In [15]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from Flask import Flask, request, jsonify 


In [16]:
housing_data = pd.read_csv('AmesHousing.csv')
housing_data_cleaned = housing_data.drop(columns=['PID', 'Order'])

In [17]:
label_encoders = {}
for column in housing_data_cleaned.columns:
    if housing_data_cleaned[column].dtype == 'object':
        housing_data_cleaned[column].fillna(housing_data_cleaned[column].mode()[0], inplace=True)
        le = LabelEncoder()
        housing_data_cleaned[column] = le.fit_transform(housing_data_cleaned[column])
        label_encoders[column] = le
    else:
        housing_data_cleaned[column].fillna(housing_data_cleaned[column].median(), inplace=True)


/var/folders/b2/232rtjw13gnfkrkttk2nc5v40000gn/T/ipykernel_10439/2844014058.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  housing_data_cleaned[column].fillna(housing_data_cleaned[column].median(), inplace=True)
/var/folders/b2/232rtjw13gnfkrkttk2nc5v40000gn/T/ipykernel_10439/2844014058.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate ob

In [18]:
X = housing_data_cleaned.drop(columns=['SalePrice'])
y = housing_data_cleaned['SalePrice']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

filename = 'model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 15917.266774744025


In [ ]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)


app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = np.array([list(data.values())])
    prediction = model.predict(features)
    return jsonify({'prediction': prediction[0]})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/furkanay/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/furkanay/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/furkanay/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/furkanay/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/furkanay/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shel

SystemExit: 1

/Users/furkanay/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
